<h1 align="center">Topic modeling on Medium articles</h1>

## Minimum setup and libraries required

In [1]:
## All minimum setup and libraries required

import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## TENSORFLOW        
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer   ## Generate dictionary of word encodings
from tensorflow.keras.preprocessing.sequence import pad_sequences

## GENSIM and NLTK
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

# SKLEARN
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

print("Tensorflow\t-\t",tf.__version__)
print("NLTK\t\t-\t",nltk.__version__)
print("Gensim\t\t-\t",nltk.__version__)

/kaggle/input/medium-articles-with-content/Medium_AggregatedData.csv
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Tensorflow	-	 2.3.0
NLTK		-	 3.2.4
Gensim		-	 3.2.4


In [2]:
path = "../input/medium-articles-with-content/Medium_AggregatedData.csv"
dataframe_full = pd.read_csv(path)
dataframe_imp = pd.read_csv(path)
print("Dataset have been read")

Dataset have been read


In [3]:
dataframe_full.head()

,audioVersionDurationSec,codeBlock,codeBlockCount,collectionId,createdDate,createdDatetime,firstPublishedDate,firstPublishedDatetime,imageCount,isSubscriptionLocked,...,slug,name,postCount,author,bio,userId,userName,usersFollowedByCount,usersFollowedCount,scrappedDate
0,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,blockchain,Blockchain,265164.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
1,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,samsung,Samsung,5708.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
2,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,it,It,3720.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
3,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,technology,Technology,166125.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104
4,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,robotics,Robotics,9103.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104


In [4]:
x = dataframe_full['name'][10]
y = dataframe_full['publicationdescription'][15]
print(x)
print(y)
print(dataframe_full.shape)
print(dataframe_full['name'][10])
print(dataframe_full['name'][11])
print(dataframe_full['name'][12])
print(dataframe_full['title'][10])

Big Data Training Mumbai
Non-obvious meditation advice from people on the battlefront of daily creation
(279577, 50)
Big Data Training Mumbai
Robotics
Meditation
Ascent of data Science, SAS and Big data Analyst Trainings Programs


# Step_1: Preprocessing and cleaning

## There are ~300000 entries

In [5]:
print(dataframe_full.columns)

Index(['audioVersionDurationSec', 'codeBlock', 'codeBlockCount',
       'collectionId', 'createdDate', 'createdDatetime', 'firstPublishedDate',
       'firstPublishedDatetime', 'imageCount', 'isSubscriptionLocked',
       'language', 'latestPublishedDate', 'latestPublishedDatetime',
       'linksCount', 'postId', 'readingTime', 'recommends',
       'responsesCreatedCount', 'socialRecommendsCount', 'subTitle',
       'tagsCount', 'text', 'title', 'totalClapCount', 'uniqueSlug',
       'updatedDate', 'updatedDatetime', 'url', 'vote', 'wordCount',
       'publicationdescription', 'publicationdomain',
       'publicationfacebookPageName', 'publicationfollowerCount',
       'publicationname', 'publicationpublicEmail', 'publicationslug',
       'publicationtags', 'publicationtwitterUsername', 'tag_name', 'slug',
       'name', 'postCount', 'author', 'bio', 'userId', 'userName',
       'usersFollowedByCount', 'usersFollowedCount', 'scrappedDate'],
      dtype='object')


### Required columns:

* language

* subTitle

* tagsCount

* text

* title

* url

* wordCount

* publicationdescription

* tag_name

* name

### Primary columns:

* subTitle

* text

* title

In [6]:
required_col = ['language','subTitle','tagsCount','text','title','url','wordCount','publicationdescription'
               ,'tag_name','name']
most_imp_col = ['subTitle','text','title']

## Removing Null rows

In [9]:
# Number of rows dropped after removing null value rows

print(dataframe_imp.shape)
dataframe_imp.dropna(how = 'all')
print(dataframe_imp.shape)

(279577, 50)
(279577, 50)


So missing rows, so no null values

## Available languages

In [7]:
# article_titles = dataframe_full['title']
# art_grp_1 = article_titles[16:25]
# print(art_grp_1)
print(dataframe_full.language.unique())

['en' 'th' 'ja' 'zh' 'ru' 'pt' 'es' 'zh-Hant' 'id' 'my' 'de' 'tr' 'fr'
 'ko' 'it' 'lo' 'un' 'vi' 'cs' 'sk' 'is' 'sv' 'bn' 'mn' 'da' 'no' 'bg'
 'ar' 'pl' 'nl' 'ro' 'ca' 'hu' 'hi' 'ka' 'el' 'ms' 'uk' 'si' 'sr' 'lt'
 'la' 'fa' 'ml' 'sl' 'mr' 'az' 'lv' 'te' 'mk' 'nn' 'fi']


## Number of English rows

In [8]:
english_titles = dataframe_full[dataframe_full['language'] == 'en']
# english_titles.head()
print(english_titles.shape)

(257655, 50)


## Removing Non - English Rows and Unnecessary Columns

In [10]:
## After dropping non-english and columns that are not required really

dataframe_imp.drop(dataframe_imp[dataframe_imp['language'] != 'en'].index, inplace = True)

dataframe_imp = dataframe_imp.drop(['audioVersionDurationSec', 'codeBlock', 'codeBlockCount',
       'collectionId', 'createdDate', 'createdDatetime', 'firstPublishedDate',
       'firstPublishedDatetime', 'imageCount', 'isSubscriptionLocked',
       'language', 'latestPublishedDate', 'latestPublishedDatetime',
       'linksCount', 'postId', 'readingTime', 'recommends',
       'responsesCreatedCount', 'socialRecommendsCount','tagsCount','totalClapCount', 'uniqueSlug',
       'updatedDate', 'updatedDatetime', 'url', 'vote', 'wordCount',
       'publicationdescription', 'publicationdomain',
       'publicationfacebookPageName', 'publicationfollowerCount',
       'publicationname', 'publicationpublicEmail', 'publicationslug',
       'publicationtags', 'publicationtwitterUsername', 'tag_name', 'slug',
       'name', 'postCount', 'author', 'bio', 'userId', 'userName',
       'usersFollowedByCount', 'usersFollowedCount', 'scrappedDate'], axis=1)

dataframe_imp['index'] = dataframe_imp.index

dataframe_imp.shape

(257655, 4)

## Reduced dataset with 3 columns and ~20000 rows for reference
### Run these cells only to produce and export the reduced and compact dataset

In [11]:
## Run these cell to export the new trimmed down dataset

# dataframe_imp.to_csv("medium_dataset.csv",sep=",")
# new_ds_size = os.stat("medium_dataset.csv").st_size
# new_ds_size = new_ds_size / 1000000
# print("New dataset size in MB = ",new_ds_size)

***Choose one among the two in the cell below***

In [12]:
## Keeping 170000 rows greatly reduces the dataset size to around 100MB

very_reduced_dataset = dataframe_imp[:17000]
# very_reduced_dataset.to_csv("very_reduced_dataset.csv",sep=",")
# print("New dataset size in MB = ",os.stat("very_reduced_dataset.csv").st_size / 1000000)

# from IPython.display import FileLink
# FileLink(r'very_reduced_dataset.csv')

***After dropping all the non-english rows and all non essential columns `dataframe_imp` is the required dataframe***

In [13]:
dataframe_imp.head()

,subTitle,text,title,index
0,A major private IT company implements blockcha...,"Private Business, Government and Blockchain\n\...","Private Business, Government and Blockchain",0
1,A major private IT company implements blockcha...,"Private Business, Government and Blockchain\n\...","Private Business, Government and Blockchain",1
2,A major private IT company implements blockcha...,"Private Business, Government and Blockchain\n\...","Private Business, Government and Blockchain",2
3,Introduction,EPQ draft 1 (4844 words)\nhttps://upload.wikim...,EPQ draft 1 (4844 words),3
4,Introduction,EPQ draft 1 (4844 words)\nhttps://upload.wikim...,EPQ draft 1 (4844 words),4


In [14]:
print(dataframe_imp.title[15])
print(dataframe_imp.subTitle[15])
# print(dataframe_imp.text[15])      ## Text is too huge to be displayed
print(dataframe_imp.index[15])

Can a robot love us better than another human can?
I discussed this with Michelle Tsng on my Podcast “Crazy Wisdom”.
15


## Perform lemmatization and stem preprocessing steps on the data set


In [15]:
## Stemmer initialization for english
stemmer = SnowballStemmer("english")

In [16]:
## Functions for lemmatization, removal of Stopwords

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [17]:
### Code to check the function

## Run this for faster execution time wiht less acuracy
doc_sample = very_reduced_dataset[very_reduced_dataset['index'] == 1000].values[0][2]

## Run this for slower execution but better accuracy
# doc_sample = dataframe_imp[dataframe_imp['index'] == 1000].values[0][2]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
    
print(dataframe_imp[dataframe_imp['index'] == 1000].values[0][2])
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
Machine Learning Made Easy: What it is and How it Works
['Machine', 'Learning', 'Made', 'Easy:', 'What', 'it', 'is', 'and', 'How', 'it', 'Works']


 tokenized and lemmatized document: 
['machin', 'learn', 'easi', 'work']


## Processed titles

In [18]:
## Use this to reduce the training time at the cost of loss of rows and some accuracy.
title_list = very_reduced_dataset['title'].astype(str)

## Use this for higher model accuracy but very slow operating time. 
# title_list = dataframe_imp['title'].astype(str)   ## using astype(str) eliminates the floting type error
title_list.describe()

count                           17000
unique                           4436
top       10 new things to read in AI
freq                               24
Name: title, dtype: object

In [19]:
## The titles are preprocessed and saved into processd_titles
## The map function applies the preprocess method on each of the list entries

processed_titles = title_list.map(preprocess)
processed_titles[30:40]

34    [meta, model, meta, meta, model, deep, learn]
35    [meta, model, meta, meta, model, deep, learn]
36               [tip, data, scienc, team, succeed]
37               [tip, data, scienc, team, succeed]
38               [tip, data, scienc, team, succeed]
39                                   [trust, trust]
40                                   [trust, trust]
41                                   [trust, trust]
42                                   [trust, trust]
43                                   [trust, trust]
Name: title, dtype: object